In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [12]:
# Assuming that you have your images in a directory structure where each category has its own folder
data_dir = './../../Training/augmentation1/'  # Replace with the path to your dataset directory

# Image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 8

# Data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # using 20% of the data for validation
)

In [13]:
# Train dataset generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # set as training data
)

# Validation dataset generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # set as validation data
)

Found 924 images belonging to 6 classes.
Found 228 images belonging to 6 classes.


In [14]:
# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Adjust the number of neurons to match the number of classes
])

In [15]:
# Model compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint( "./../../models/new-model.h5" , monitor='val_accuracy', save_best_only=True)

In [17]:

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/100
115/115 [==============================] - ETA: 0s - loss: 1.4332 - accuracy: 0.4236

/home/antoine/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


115/115 [==============================] - 112s 960ms/step - loss: 1.4332 - accuracy: 0.4236 - val_loss: 1.1801 - val_accuracy: 0.5089
Epoch 2/100
115/115 [==============================] - 126s 1s/step - loss: 0.9430 - accuracy: 0.6070 - val_loss: 0.8893 - val_accuracy: 0.6250
Epoch 3/100
115/115 [==============================] - 120s 1s/step - loss: 0.8853 - accuracy: 0.6539 - val_loss: 0.8753 - val_accuracy: 0.6830
Epoch 4/100
115/115 [==============================] - 120s 1s/step - loss: 0.7882 - accuracy: 0.6889 - val_loss: 0.8086 - val_accuracy: 0.6920
Epoch 5/100
115/115 [==============================] - 118s 1s/step - loss: 0.7589 - accuracy: 0.6812 - val_loss: 1.0242 - val_accuracy: 0.6116
Epoch 6/100
115/115 [==============================] - 131s 1s/step - loss: 0.6690 - accuracy: 0.7227 - val_loss: 0.6355 - val_accuracy: 0.7946
Epoch 7/100
115/115 [==============================] - 130s 1s/step - loss: 0.5521 - accuracy: 0.7784 - val_loss: 0.5665 - val_accuracy: 0.7812
E

: 

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')